<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/modelToDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##モデルをDB化

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import IPython
#bz2ファイル解凍
#!bunzip2 -k -q "./drive/My Drive/NLP/20170201.tar.bz2"

#tarファイル解凍
!tar -xvf "./drive/My Drive/NLP/20170201.tar"

entity_vector/
entity_vector/entity_vector.model.txt
entity_vector/entity_vector.model.bin


In [4]:
from gensim.models import KeyedVectors

#解凍時のみ
model_dir = 'entity_vector/entity_vector.model.bin'
model = KeyedVectors.load_word2vec_format(model_dir, binary=True,limit=500000)

#解凍後確認
results = model.most_similar(positive=[u'数学'])
for result in results:
  print(result)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


('[数学]', 0.8441122770309448)
('物理学', 0.8005046844482422)
('算術', 0.7927638292312622)
('[幾何学]', 0.7776163220405579)
('[物理学]', 0.7547159194946289)
('[代数学]', 0.7514173984527588)
('代数学', 0.7450574636459351)
('[解析学]', 0.741286039352417)
('[確率論]', 0.7390509843826294)
('[数論]', 0.7337667942047119)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [5]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 3.1MB/s 


##語彙-対義語候補 対応CSV作成

In [13]:
import re
import pandas as pd

cnt = 1
model_vocab_list = []

for words, v in model.vocab.items():

  words = words.replace('_', '')
  
  #固有名詞用の括弧を削除
  words = words.replace('[', '').replace(']', '')

  #補足情報の_を削除
  words = words.replace("_","")
  #括弧文字を抽出
  regex = re.compile(".*?\((.*?)\)")
  #括弧文字をlist型で返却
  ret_list = re.findall(regex, words)

  #括弧文字がある場合は無効化
  if len(ret_list) > 0: 
    words = words.replace("(" + ret_list[0] + ")",'')
    model_vocab_list.append(words)
  else:
    model_vocab_list.append(words)
    
  if cnt % 50000 == 0:
    print(cnt,"件 完了");
    
  cnt = cnt + 1

#重複削除
model_vocab_uni_list= list(set(model_vocab_list))
  
#保存
df = pd.DataFrame(model_vocab_uni_list)
df.to_csv('./drive/My Drive/NLP/model_vocab_list.csv', header=False, index=False)

50000 件 完了
100000 件 完了
150000 件 完了
200000 件 完了
250000 件 完了
300000 件 完了
350000 件 完了
400000 件 完了
450000 件 完了
500000 件 完了


In [14]:
import pandas as pd

csv_dir = './drive/My Drive/NLP/model_vocab_list.csv'
df = pd.read_csv(csv_dir,names=['words'])
df = df.sort_values('words', ascending=True)

print(df.count())
print(df[df["words"]=="ゆず"])

words    425495
dtype: int64
       words
360410    ゆず


In [0]:
import MeCab
#括弧記号無効化
import lib_delete_brackets

def search_similar_texts(p_words):
  pass

def wordRevChange(words,gyaku,inherent_words,model,tokenizer):

  word_cng_list = []

  #**************************************************************************
  #そのまま対義化
  #**************************************************************************
  asitis_cng_words1 = ''
  asitis_cng_words2 = ''
  try:
    for i in range(2):
      rvs_wd = model.most_similar(positive=[inherent_words,gyaku])[i][0]
      if inherent_words.replace('[',"").replace(']',"") != rvs_wd.replace('[',"").replace(']',""):
        asitis_cng_words1 = model.most_similar(positive=[inherent_words,gyaku])[i][0]
        asitis_cng_words2 = model.most_similar(positive=[inherent_words,gyaku])[i+1][0]
        break
      else:
        asitis_cng_words = words
  except KeyError as error:
    #辞書に登録の無い単語の場合
    search_similar_texts(inherent_words)

  #**************************************************************************
  #形態素分析後に対義化
  #**************************************************************************
  node = tokenizer.parseToNode(words)

  while node:
    cut_wd = node.surface

    if node.feature.split(",")[0] == u"名詞":
      try:
        rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
        for i in range(2):
          rvs_wd = rvs_wd_list[i][0]
          if cut_wd != rvs_wd.replace('[',"").replace(']',""):
            #◆結合
            word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
            break
      except KeyError as error:
        #辞書に登録の無い単語の場合
        search_similar_texts(cut_wd)

    elif node.feature.split(",")[0] == u"形容詞":
      cut_wd = node.feature.split(",")[6]

      try:
        rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
        for i in range(2):
          rvs_wd = rvs_wd_list[i][0]
          if cut_wd != rvs_wd.replace('[',"").replace(']',""):
            #◆結合
            word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
            break
      except KeyError as error:
        #辞書に登録の無い単語の場合
        search_similar_texts(cut_wd)

    elif node.feature.split(",")[0] == u"動詞":
      cu_wd = node.feature.split(",")[6]

      try:
        rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
        for i in range(2):
          rvs_wd = rvs_wd_list[i][0]
          if cut_wd != rvs_wd.replace('[',"").replace(']',""):
            #◆結合
            word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
            break
      except KeyError as error:
        #辞書に登録の無い単語の場合
        search_similar_texts(cut_wd)

    else:
      #◆結合
      word_cng_list.append(cut_wd.replace('[',"").replace(']',""))

    node = node.next

  asitis_cng_words1 = asitis_cng_words1.replace('[',"").replace(']',"").replace('_',"")
  asitis_cng_words2 = asitis_cng_words2.replace('[',"").replace(']',"").replace('_',"")

  result_list = []

  result_list.append(words)
  #result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words1))
  #result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words2))
  result_list.append(asitis_cng_words1)
  result_list.append(asitis_cng_words2)
  result_list.append(''.join(word_cng_list))

  return result_list


In [0]:
import re
import pandas as pd
import MeCab
#対義語生成
#import lib_wordRevChange as lw

tokenizer = MeCab.Tagger("-Ochasen")

cnt = 1
save_list = []

#列[words]をnumpyに変換
val = df.words.values

for idx in range(df.shape[0]):

  words = str(val[idx])
  
  if len(words) <= 10:
    if words.isdecimal() is False:
      if '・' not in words:
        gyaku = u"逆"
        inherent_words = '[' + words + ']'  
        rev_list = wordRevChange(words,gyaku,inherent_words,model,tokenizer)
        save_list.append(rev_list)

  if cnt % 1000 == 0:
    print(cnt,"件 終了")
    #Gドライブ再マウント
    drive.mount('/content/drive')  
    #保存
    df2 = pd.DataFrame(save_list)
    df2.to_csv('./drive/My Drive/NLP/modelToCsv.csv', header=False, index=False)
    
  cnt = cnt + 1

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
rt = model.most_similar(positive=["[世界に一つだけの花]"])
rt

In [0]:
df2